# Slit Width Trade Book
### Bandpass vs. SNR vs. Along-Track Spatial Resolution

In [1]:
# stdlib
import math
from pathlib import Path

# external
import astropy.units as unit
import numpy as np
import pandas as pd
import plot
from IPython.display import display

# project
from architect import luts
from architect.libs import utillib
from architect.systems.optical import (
    diffractors,
    foreoptics,
    lenses,
    masks,
    sensors,
    spectrometers,
)

## Setup
Run this section onwards when a parameter is updated.

### Parameters

In [2]:
# constants
target_wavelength = 1300 * unit.nm
lower_wavelength = 900 * unit.nm
upper_wavelength = 1700 * unit.nm
beam_diameter = 12 * unit.mm
fringe_frequency = 600 / unit.mm
# skew_angle = np.array([0, 0]) * unit.deg
# orbital_altitude = 550 * unit.km

# variables
widths = np.arange(start=10, stop=1000, step=1) * unit.micrometer

### Instantiation

In [3]:
# components
sensor = sensors.TauSWIR()
foreoptic = foreoptics.Foreoptic(
    focal_length=100 * unit.mm, diameter=35 * unit.mm, image_diameter=10 * unit.mm
)
diffractor = diffractors.TransmissiveDiffractor(fringe_frequency=fringe_frequency)
collimator = lenses.Lens(focal_length=13 * unit.mm)
radiance = luts.load("atmosphere/radiance_min")

## Pipeline
The computational graph

In [4]:
bandpass = []
snr = []
spatial_resolution = []

for width in widths:
    # Modify the slit width and then reinstantiate the HyperspectralImager
    slit = masks.RectSlit(size=(width, 7.68 * unit.mm))
    payload = spectrometers.HyperspectralImager(
        sensor=sensor,
        foreoptic=foreoptic,
        slit=slit,
        diffractor=diffractor,
        collimator=collimator,
    )

    # Determine the bandpass with the new slit width
    bandpass_measurement = payload.get_bandpass().to(unit.nm)
    bandpass.append(bandpass_measurement.value)

    # Determine the SNR for the new slit width
    snr_measurment = payload.get_signal_to_noise(
        radiance=radiance, wavelength=target_wavelength
    )
    snr.append(snr_measurment.value)

    # Determine the Spatial Resolution for the new slit
    # spatial_resolution_measurment = payload.get_spatial_resolution(wavelength=target_wavelength, target_distance=orbital_altitude,skew_angle=skew_angle[0],)
    # spatial_resolution.append(spatial_resolution_measurment.to(unit.km).value)

## Plots

In [5]:
print(spatial_resolution)
widths_label = "Slit Width (microns)"
bandpass_label = "Spectral Bandpass (nm)"
snr_label = "SNR"
spatial_resolution_label = "Spatial Resolution (km)"
data = {
    widths_label: widths[:],
    bandpass_label: bandpass[:],
    snr_label: snr[:],
}

df = pd.DataFrame.from_dict(data=data)
fig = plot.line(
    df=df,
    x=widths_label,
    y=[bandpass_label, snr_label],
    title="Slit Width Trade Off",
    dark=True,
)
fig.show()
display(df)

[]


,Slit Width (microns),Spectral Bandpass (nm),SNR
0,10.0,1.282051,0.503009
1,11.0,1.410256,0.553310
2,12.0,1.538462,0.603610
3,13.0,1.666667,0.653910
4,14.0,1.794872,0.704210
...,...,...,...
985,995.0,127.564103,49.996391
986,996.0,127.692308,50.046584
987,997.0,127.820513,50.096778
988,998.0,127.948718,50.146972
